In [1]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    cohen_kappa_score,
    classification_report,
)
import warnings

warnings.filterwarnings("ignore")

In [2]:
# Путь к объединенным CSV-файлам
combined_path = "combined_csvs"
combined_files = [f for f in os.listdir(combined_path) if f.endswith(".csv")]

# Сортировка файлов по числу пакетов (например, _250_)
combined_files.sort(key=lambda f: int(re.search(r"(\d+)_packets", f).group(1)))

# Словарь моделей с параметром verbose
models = {
    "Random Forest": RandomForestClassifier(random_state=42, verbose=1, n_jobs=-1),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42, verbose=1),
    "SVM": LinearSVC(verbose=1, random_state=42),
    "k-NN": KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
    "Logistic Regression": LogisticRegression(
        solver="saga",
        multi_class="ovr",
        class_weight="balanced",
        max_iter=1000,
        verbose=1,
        n_jobs=-1,
        random_state=42,
    ),
}

# Создание папки для промежуточных файлов
intermediate_path = "binary_class_models_results"
os.makedirs(intermediate_path, exist_ok=True)

# Итоговый список для всех результатов
all_results = []

# Проход по каждому CSV-файлу
for combined_file in tqdm(combined_files, desc="Обработка файлов"):
    df = pd.read_csv(os.path.join(combined_path, combined_file))
    X = df.drop(columns=["Label"])
    y = df["Label"]

    # Извлечение размера пакетов из имени файла
    packet_size = int(re.search(r"(\d+)_packets", combined_file).group(1))

    # Разделение данных и нормализация в отдельном блоке try
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y
        )
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    except ValueError as e:
        print(f"  ! Ошибка при подготовке данных для файла '{combined_file}': {e}")
        continue

    # Проход по каждой модели
    for model_name, model in models.items():
        results = {
            "Model": model_name,
            "File": combined_file,
            "Packet Size": packet_size,
        }

        print(f"  - Обучение модели {model_name}")
        try:
            # Обучение модели с выводом прогресса
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Расчёт метрик
            results["Accuracy"] = accuracy_score(y_test, y_pred)
            results["Macro F1 Score"] = f1_score(y_test, y_pred, average="macro")
            results["Micro F1 Score"] = f1_score(y_test, y_pred, average="micro")
            results["Weighted F1 Score"] = f1_score(y_test, y_pred, average="weighted")
            results["Macro Precision"] = precision_score(
                y_test, y_pred, average="macro"
            )
            results["Micro Precision"] = precision_score(
                y_test, y_pred, average="micro"
            )
            results["Cohen's Kappa"] = cohen_kappa_score(y_test, y_pred)

            # Отчёт по каждому классу
            class_report = classification_report(y_test, y_pred, output_dict=True)
            for label, metrics in class_report.items():
                if isinstance(metrics, dict):
                    results[f"Precision_{label}"] = metrics["precision"]
                    results[f"Recall_{label}"] = metrics["recall"]
                    results[f"F1_{label}"] = metrics["f1-score"]

        except ValueError as e:
            # Пропуск метрик при ошибке
            results.update(
                {
                    "Accuracy": "0.0",
                    "Macro F1 Score": "0.0",
                    "Micro F1 Score": "0.0",
                    "Weighted F1 Score": "0.0",
                    "Macro Precision": "0.0",
                    "Micro Precision": "0.0",
                    "Cohen's Kappa": "0.0",
                }
            )
            print(
                f"  ! Ошибка обучения модели '{model_name}' для файла '{combined_file}': {e}"
            )

        # Добавление результата
        all_results.append(results)

        # Промежуточный CSV для модели
        model_results_df = pd.DataFrame([results])
        model_results_file = os.path.join(
            intermediate_path, f"{model_name}_multiclass_results.csv"
        )

        # Если файл существует, добавляем данные
        if os.path.exists(model_results_file):
            model_results_df.to_csv(
                model_results_file, mode="a", index=False, header=False, float_format="%.3f"
            )
        else:
            model_results_df.to_csv(model_results_file, index=False, float_format="%.3f")

# Итоговый файл с результатами всех моделей
final_results_df = pd.DataFrame(all_results)
final_results_df.to_csv("final_multiclass_models_results.csv", index=False, float_format="%.3f")
print("Итоговый файл сохранён в 'final_multiclass_models_results.csv'")

Обработка файлов:   0%|          | 0/9 [00:00<?, ?it/s]

  - Обучение модели Random Forest


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   12.5s finished
[Parallel(n_jobs=28)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=28)]: Done 100 out of 100 | elapsed:    0.3s finished


  - Обучение модели Gradient Boosting
      Iter       Train Loss   Remaining Time 
         1           1.2031            1.39m
         2           1.0769            1.30m
         3           0.9718            1.29m
         4           0.8838            1.25m
         5           0.8094            1.23m
         6           0.7448            1.20m
         7           0.6895            1.18m
         8           0.6417            1.16m
         9           0.6002            1.14m
        10           0.5640            1.12m
        20           0.3713           58.57s
        30           0.3090           51.55s
        40           0.2819           44.27s
        50           0.2661           36.75s
        60           0.2527           29.34s
        70           0.2437           22.08s
        80           0.2375           14.77s
        90           0.2314            7.43s
       100           0.2280            0.00s
  - Обучение модели SVM
[LibLinear]  - Обучение модели k-NN
 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 28 concurrent workers.


...

  - Обучение модели Random Forest


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=28)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=28)]: Done 100 out of 100 | elapsed:    0.0s finished


  - Обучение модели Gradient Boosting
      Iter       Train Loss   Remaining Time 
         1           1.1712            0.30s
         2           1.0184            0.24s
         3           0.8918            0.26s
         4           0.7852            0.24s
         5           0.6944            0.23s
         6           0.6163            0.22s
         7           0.5487            0.23s
         8           0.4898            0.22s
         9           0.4383            0.21s
        10           0.3930            0.22s
        20           0.1428            0.18s
        30           0.0546            0.16s
        40           0.0219            0.14s
        50           0.0088            0.12s
        60           0.0032            0.10s
        70           0.0012            0.07s
        80           0.0004            0.05s
        90           0.0002            0.03s
       100           0.0001            0.00s
  - Обучение модели SVM
[LibLinear]  - Обучение модели k-NN
 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 28 concurrent workers.
Обработка файлов: 100%|██████████| 9/9 [06:35<00:00, 43.91s/it]

convergence after 298 epochs took 0 seconds
Итоговый файл сохранён в 'final_multiclass_models_results.csv'


In [3]:
# Создание папки для модифицированных промежуточных файлов
intermediate_path_modified = "binary_class_models_results_modified/"
os.makedirs(intermediate_path_modified, exist_ok=True)

# Путь к промежуточным файлам
intermediate_path = "binary_class_models_results/"
model_results_files = [f for f in os.listdir(intermediate_path) if f.endswith("_multiclass_results.csv")]

# Создаем итоговый DataFrame для всех моделей
final_results = []

# Процесс обработки каждого промежуточного файла модели
for model_results_file in model_results_files:
    # Чтение промежуточного CSV файла
    df = pd.read_csv(os.path.join(intermediate_path, model_results_file))
    # display(df)

    # Получаем список метрик, исключая "Packet Size" и "Model"
    metrics_columns = [col for col in df.columns if col not in ['Packet Size', 'Model', 'File']]

    # Определение лучшей выборки для каждой метрики
    best = {}
    counts = {}

    # Добавление столбца Counts для каждой выборки
    max1_counts_column = []
    for index, row in df.iterrows():
        max1_count = sum(1 for col in metrics_columns if row[col] == df[col].max())  # Считаем, сколько раз максимальное значение встречается
        max1_counts_column.append(max1_count)

    df["max 1"] = max1_counts_column

    # Для каждой метрики находим наилучшие выборки
    for col in metrics_columns:  # Перебираем только метрики
        max_value = df[col].max()  # Находим максимальное значение для метрики
        best[col] = df[df[col] == max_value]["Packet Size"].tolist()  # Сохраняем размеры выборок с максимальным значением

        # Подсчитаем, сколько раз каждая выборка была максимальной для этой метрики
        counts[col] = df[col].value_counts().get(max_value, 0)  # Подсчитаем сколько раз максимальное значение встречается

    # Строка Best
    best_row = {"Packet Size": "Best"}
    for col in metrics_columns:
        best_row[col] = best[col]  # Присваиваем лучшие выборки для каждой метрики

    # Добавление строки Best в DataFrame
    df_best = pd.DataFrame([best_row])

    # Добавление строки Best в DataFrame
    df = pd.concat([df, df_best], ignore_index=True)
    
    model_results_file_modified = os.path.join(
            intermediate_path_modified, f"{os.path.splitext(model_results_file)[0]}_modified.csv"
        )

    # Запись результатов в новый CSV файл
    df.to_csv(model_results_file_modified, index=False)